# County Supplemental Data

In [2]:
""" Notebook Setup/Imports
"""

# Project path setup
from pathlib import Path
project_dir = Path(__name__).resolve().parents[1]

# Library import
import pandas as pd
import numpy as np
import plotly as py
py.tools.set_credentials_file(username="NoahtTristan", api_key="A3pWAtQEqacwCMtXMNvR")
import plotly.figure_factory as ff

# Print strings as markdown
from IPython.display import Markdown

# Data variables from previous notebook
dataset = pd.read_pickle(Path.joinpath(project_dir, "data/processed/1_data.pickle"))
data_dictionary = pd.read_pickle(Path.joinpath(project_dir, "data/processed/1_data_dictionary.pickle"))

## Additional County Data
> https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/

In [28]:
education = pd.read_csv("../data/external/Education.csv",
                                 encoding='latin-1').set_index("FIPS Code")

In [32]:
poverty = pd.read_csv("../data/external/PovertyEstimates.csv",
                                 encoding='latin-1').set_index("FIPStxt")

In [36]:
unemployment = pd.read_excel("../data/external/Unemployment.xls", skiprows=7,
                                 encoding='latin-1').set_index("FIPStxt")

In [65]:
ed_keep_cols = []
ed_cols = education.columns.values.tolist()
for col in ed_cols:
    if (col.find("1970") < 0 and col.find("1980") < 0 
        and col.find("1990") < 0 and col.find("2000") < 0
       and col.find("2003") < 0 and col != 'State'
       and col != "Area name" and col.find("2013 ") < 0
       and col.find("Unnamed") < 0):
        ed_keep_cols.append(col)

In [67]:
education = education[ed_keep_cols]

In [77]:
pov_keep_cols = []
pov_cols = poverty.columns.values.tolist()
for col in pov_cols:
    if (col.find("1970") < 0 and col.find("1980") < 0 
        and col.find("1990") < 0 and col.find("2000") < 0
       and col.find("2003") < 0 and col != 'State'
       and col.find("Area") < 0 and col.find("2013 ") < 0
       and col.find("Unnamed") < 0 and col.find("_2013") < 0):
        pov_keep_cols.append(col)

In [80]:
poverty = poverty[pov_keep_cols]

In [91]:
unemployment_keep_cols = []
unemployment_cols = unemployment.columns.values.tolist()
for col in unemployment_cols:
    if (col.find("1970") < 0 and col.find("1980") < 0 
        and col.find("1990") < 0 and col.find("2000") < 0
       and col.find("2003") < 0 and col != 'State'
       and col.find("Area") < 0 and col.find("2013 ") < 0
       and col.find("Unnamed") < 0 and col.find("_2013") < 0
       and col.find("_2007") < 0 and col.find("_2008") < 0
       and col.find("_2009") < 0 and col.find("_2010") < 0
       and col.find("_2011") < 0 and col.find("_2012") < 0
       and col.find("_2014") < 0 and col.find("_2016") < 0
       and col.find("_2017") < 0):
        unemployment_keep_cols.append(col)

In [93]:
unemployment = unemployment[unemployment_keep_cols]

In [108]:
county_data = education.join(
    poverty,rsuffix="_").join(
    unemployment,rsuffix="_")

In [110]:
full_dataset = dataset.set_index("index").join(county_data)

In [121]:
pov_dd = pd.read_excel("../data/external/PovertyEstimates.xls",sheet_name=1).loc[:33]

In [127]:
pov_dd = pov_dd.rename({
    "Column variable name":"Variable Code",
    "Description":"Variable Name"
},axis="columns").drop("Notes",axis="columns")

In [131]:
data_dictionary = pd.concat([data_dictionary,pov_dd],sort=False)

In [133]:
un_dd = pd.read_excel("../data/external/Unemployment.xls",sheet_name=1)

In [136]:
un_dd = un_dd.loc[:51].rename({
    "Variable":"Variable Code",
    "Description":"Variable Name"
},axis="columns")

In [138]:
data_dictionary = pd.concat([data_dictionary,un_dd],sort=False)

In [142]:
# Save data
dataset.to_pickle("../data/production/data.pickle")
data_dictionary.to_pickle("../data/production/data_dictionary.pickle")

In [143]:
# Save data
dataset.to_csv("../data/production/data.csv")
data_dictionary.to_csv("../data/production/data_dictionary.csv")